In [3]:
import torch

In [4]:
print(torch.__version__)

2.9.0+cu126


In [5]:
print(torch.cuda.is_available())

True


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip -q data.zip

--2025-12-02 08:20:58--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T09%3A13%3A29Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T08%3A12%3A41Z&ske=2025-12-02T09%3A13%3A29Z&sks=b&skv=2018-11-09&sig=rhKAJkoYNxP5FHNjEgRl5RY%2BKJoLVI6bTBlVS9nwCIk%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDY2NTQ1OSwibmJmIjoxNzY0NjYzNjU5LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

In [8]:
import numpy as np

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [9]:
import torch.nn as nn
import torch.nn.functional as F

In [28]:
class HairClassifier(nn.Module):
    def __init__ (self):
      super().__init__()
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)

      self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

      self.flatten_size = 32 * 99 * 99

      self.fc1 = nn.Linear(self.flatten_size, 64)

      self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, self.flatten_size)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

model = HairClassifier().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()




In [29]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


In [30]:
from torchvision import transforms
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [31]:
from torch.utils.data import DataLoader
import torchvision

train_dataset = torchvision.datasets.ImageFolder('./data/train', transform=train_transforms)
validation_dataset = torchvision.datasets.ImageFolder('./data/test', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False, num_workers=2)

print(f"Train batch: {len(train_dataset)}")
print(f"Validation batch: {len(validation_dataset)}")

Train batch: 800
Validation batch: 201


In [32]:
model = HairClassifier().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # Apply sigmoid for accuracy calculation
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation part (kept for completeness, though Q3 only asks for train acc)
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Acc: {epoch_acc:.4f}, Val Acc: {val_epoch_acc:.4f}")

Starting training...
Epoch 1/10, Train Acc: 0.6375, Val Acc: 0.5871
Epoch 2/10, Train Acc: 0.6987, Val Acc: 0.6517
Epoch 3/10, Train Acc: 0.7462, Val Acc: 0.6617
Epoch 4/10, Train Acc: 0.7762, Val Acc: 0.6667
Epoch 5/10, Train Acc: 0.8037, Val Acc: 0.6866
Epoch 6/10, Train Acc: 0.8225, Val Acc: 0.6716
Epoch 7/10, Train Acc: 0.8638, Val Acc: 0.7065
Epoch 8/10, Train Acc: 0.9313, Val Acc: 0.7015
Epoch 9/10, Train Acc: 0.9413, Val Acc: 0.7363
Epoch 10/10, Train Acc: 0.9437, Val Acc: 0.7264


In [34]:
median_acc = np.median(history['acc'])
std_loss = np.std(history['loss'])

print(f"Median Training Accuracy: {median_acc:.2f}")
print(f"Standard Deviation of Training Loss: {std_loss:.4f}")

Median Training Accuracy: 0.81
Standard Deviation of Training Loss: 0.1653


In [35]:
train_transforms_aug = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset_aug = torchvision.datasets.ImageFolder('./data/train', transform=train_transforms_aug)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True, num_workers=2)


In [36]:

aug_history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}

print("Continuing training with augmentation for 10 epochs...")

for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0


    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    aug_history['loss'].append(epoch_loss)
    aug_history['acc'].append(epoch_acc)


    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    aug_history['val_loss'].append(val_epoch_loss)
    aug_history['val_acc'].append(val_epoch_acc)

    print(f"Aug Epoch {epoch+1}/10, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Continuing training with augmentation for 10 epochs...
Aug Epoch 1/10, Loss: 0.6215, Acc: 0.6813, Val Loss: 0.6024, Val Acc: 0.6816
Aug Epoch 2/10, Loss: 0.5498, Acc: 0.7163, Val Loss: 0.5625, Val Acc: 0.7363
Aug Epoch 3/10, Loss: 0.5223, Acc: 0.7163, Val Loss: 0.4996, Val Acc: 0.7363
Aug Epoch 4/10, Loss: 0.5153, Acc: 0.7362, Val Loss: 0.5854, Val Acc: 0.7015
Aug Epoch 5/10, Loss: 0.5051, Acc: 0.7488, Val Loss: 0.5247, Val Acc: 0.7463
Aug Epoch 6/10, Loss: 0.4769, Acc: 0.7725, Val Loss: 0.6508, Val Acc: 0.6318
Aug Epoch 7/10, Loss: 0.4864, Acc: 0.7525, Val Loss: 0.5136, Val Acc: 0.7562
Aug Epoch 8/10, Loss: 0.4594, Acc: 0.7863, Val Loss: 0.6602, Val Acc: 0.6368
Aug Epoch 9/10, Loss: 0.4523, Acc: 0.7775, Val Loss: 0.4714, Val Acc: 0.7761
Aug Epoch 10/10, Loss: 0.4377, Acc: 0.7987, Val Loss: 0.5159, Val Acc: 0.7761


In [38]:
mean_val_loss_aug = np.mean(aug_history['val_loss'])
last_5_accuracies = aug_history['val_acc'][5:]
mean_acc = np.mean(last_5_accuracies)

print(f"Mean Validation Loss (Augmented): {mean_val_loss_aug:.2f}")
print(f"Mean Accuracy (Last 5 Epochs): {mean_acc:.2f}")

Mean Validation Loss (Augmented): 0.56
Mean Accuracy (Last 5 Epochs): 0.72
